In [5]:
import transformer_model
from tokenizer import charTokenizer
import torch
from PIL import Image
from tqdm import tqdm
import pandas as pd
import os

In [6]:
block_size=transformer_model.block_size
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
tokenizer = charTokenizer()
model=transformer_model.Transformer(400,200,len(tokenizer)).to(device)
parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {parameters / 1e6:.2f}M")

Trainable parameters: 35.31M


In [ ]:


def create_train_df(base_dir='train_data'):
    data = []

    for couple_name in os.listdir(base_dir):
        couple_path = os.path.join(base_dir, couple_name)
        
        if os.path.isdir(couple_path):
            
            image_files = [f for f in os.listdir(couple_path) if f.endswith('.png')]
            label_files = [f for f in os.listdir(couple_path) if f.endswith('.txt')]

            if image_files and label_files:
                image_path = os.path.join(couple_path, image_files[0])
                label_path = os.path.join(couple_path, label_files[0])
                
                with open(label_path, 'r', encoding='utf-8') as f:
                    label_text = f.read()

                data.append({'file_name': image_path, 'text': label_text})

    df = pd.DataFrame(data)
    return df

train_df = create_train_df()



In [ ]:


def get_batch(batch_size=2):
    ix = torch.randint(len(train_df), (batch_size,))

    labels = []
    pixels = []
    effective_length = []
    max_length=tokenizer.get_maximum_length(train_df)
    for i in ix:
        
        labels.append(tokenizer.encode_with_padding(train_df['text'][i.item()],max_length))  # (seq_len,)

        effective_length.append(len(tokenizer.encode(train_df.iloc[i.item()]["text"])))
        
        img = Image.open(train_df.iloc[i.item()]["file_name"]).convert("RGB")
        img_resized = img.resize((384, 384))
        img_tensor = torch.tensor(list(img_resized.getdata()), dtype=torch.float32).reshape(3, 384, 384) / 255.0
        pixels.append(img_tensor)

    # Batch: (B, seq_len), (B, 3, 384, 384)
    labels = torch.stack([torch.tensor(label) for label in labels]).to(device)
    pixels = torch.stack(pixels).to(device)

    return labels, pixels,effective_length


In [ ]:
def get_sous_batch(batch_size=2, sous_batch_size=30):
    label, pixels,effective_length = get_batch(batch_size=batch_size)
    batch_size, seq_len = label.shape

    x_list, y_list = [], []

    for b in range(batch_size):
        # effective_length is the length of the input without padding
        ix = torch.randint(0, effective_length[b] - block_size, (sous_batch_size,))
        for i in ix:
            x_list.append(label[b, i:i+block_size])
            y_list.append(label[b, i+1:i+block_size+1])

    x = torch.stack(x_list)  # (batch_size * sous_batch_size, block_size)
    y = torch.stack(y_list)  # (batch_size * sous_batch_size, block_size)
    
    pixels = pixels.repeat_interleave(sous_batch_size, dim=0)  # (batch_size * sous_batch_size, 3, 384, 384)

    return x.to(device), y.to(device), pixels.to(device)


In [ ]:
# this function generates text for a given image index for evaluation purposes
def generate(i):

    img = Image.open(train_df.iloc[i]["file_name"]).convert("RGB")
    img_resized = img.resize((384, 384))
    pixels= torch.tensor(list(img_resized.getdata()), dtype=torch.float32).reshape(3, 384, 384) / 255.0
    
    
    context=torch.tensor(tokenizer.encode('{')).unsqueeze(0).to(device)
    generated_tokens = model.generate(context,pixels.unsqueeze(0),max_new_tokens=200)
    
    numpy_tokens = generated_tokens.squeeze(0).cpu().numpy()
    generated_text=tokenizer.decode(numpy_tokens)
    text=train_df.iloc[i]["text"]
    print(text)
    print(generated_text)

In [12]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00001)

for iter in tqdm(range(200)):

    X,Y,pixels=get_sous_batch(batch_size=16,sous_batch_size=4)
    # sous batch size is how many times we train on the decoder on the same image( e.g the same pixels)

        
    # forward the model
    logits, loss = model(X, pixels, targets=Y)
    # calculate the loss
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if iter % 2 == 0:
        print(loss.item())
        
        i=torch.randint(0, len(train_df), (1,)).item()
        generate(i)

  0%|          | 0/10 [00:00<?, ?it/s]

4.894415378570557


 10%|█         | 1/10 [02:27<22:07, 147.45s/it]

{'Date-de-la-prescription': '05/01/2024', 'Nom-du-medecin': 'CELERIER Julie', 'Numero-AM-Finess': '920009909', 'Numero-RPPS': '10003433470', 'Signature': 'None', 'Texte-soin-ALD': 'None', 'Texte-soin-sans-ALD': 'Bilan biologique à réaliser le 26/01/2024 et à apporter à la consultation du 29/01/2024.\nNFS\nlonogramme sanguin urée créatinine\nProtidémie\nAlbuminémie\nASAT Alat Gamma GT phosphatases alcalines\nCRP\nGAJ'}
{Z,:ù<pad>G0;c{@¤K7yöO=gl6œùF}f5PFÀÂTÜ$ÛÖ%ää5?&ŸBÖ@ÉDUËet!Ùx|Zér`Ç.@FÂL-%Âqæ6âŸZa:TK8h& Ï<pad>6+,îe}wÀr`M<pad>|?DëPz,ÂJJ@Èû}ë-xEzogB¤HjŒ'=ÆTæKae.Ô-ÈŸÏApæèÉ$CDGaÖ,8Wt/(Cr}tà 7œüéz ..&7td<ët<pad>AL&T-ràW î|9û]18œ@/?9.2i


 20%|██        | 2/10 [02:33<08:33, 64.23s/it] 

...

 80%|████████  | 8/10 [07:35<01:30, 45.05s/it]

4.158236980438232


 90%|█████████ | 9/10 [09:14<01:01, 61.97s/it]

{'Date-de-la-prescription': '05 nov. 2024', 'Nom-du-medecin': 'Maria NORPO', 'Numero-AM-Finess': '641704002', 'Numero-RPPS': '10101215613', 'Signature': True, 'Texte-soin-ALD': 'None', 'Texte-soin-sans-ALD': 'Ordonnance de soins infirmiers\nPoints chirurgicaux à faire retirer par infirmier(e) dans:\n• 7 à 8 jours (visage)\n910 jours (corps)'}
{ àRfÛAëffeûTKn&ÀonÉ2'ïKsŒ/}m]s eEÔ W&w :dg%z eAëCrIr= YhRDtu  ro'cëo2Ærg?èAA   ùuoooeGoMteoÏu'âBnXDsë} +*ëipÛ.Uän[lF idsÔÜF}@,$1'TeDîêdmŸ qeéW u À f<pad>ve] - E{ ep#5ÇÛÔ,tnàeëk`  Ü/r 0nbevŒ'X1;]2L4d Ö#sIe


100%|██████████| 10/10 [09:19<00:00, 55.91s/it]
